In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from joblib import Parallel, delayed

Using Theano backend.


In [2]:
#Loading the training and testing data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [3]:
#Seed for reproducibility
np.random.seed(1338)

#Selecting 6000 random examples from the test data
test_rows = np.random.randint(0,X_test.shape[0],6000)
X_test = X_test[test_rows]
Y = y_test[test_rows]
#Converting the output to binary classification(Six=1,Not Six=0)
Y_test = Y == 6
Y_test = Y_test.astype(int)

#Selecting the 5918 examples where the output is 6
X_six = X_train[y_train == 6]
Y_six = y_train[y_train == 6]
#Selecting the examples where the output is not 6
X_not_six = X_train[y_train != 6]
Y_not_six = y_train[y_train != 6]
#Selecting 6000 random examples from the data that contains only the data where the output is not 6
random_rows = np.random.randint(0,X_not_six.shape[0],6000)
X_not_six = X_not_six[random_rows]
Y_not_six = Y_not_six[random_rows]

In [4]:
#Appending the data with output as 6 and data with output as not six
X_train = np.append(X_six,X_not_six)
#Reshaping the appended data to appropraite form
X_train = X_train.reshape(X_six.shape[0] + X_not_six.shape[0], 1, img_rows, img_cols)
#Appending the labels and converting the labels to binary classification(Six=1,Not Six=0)
Y_labels = np.append(Y_six,Y_not_six)
Y_train = Y_labels == 6 
Y_train = Y_train.astype(int)

In [5]:
#Converting the classes to its binary categorical form
nb_classes = 2
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

In [6]:
#Initializing the values for the convolution neural network
nb_epoch = 12
batch_size = 128
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
#nb_pool = 2
nb_pool = [2,3,4]
# convolution kernel size
nb_conv = 3
#Activations
activations=['relu','tanh','sigmoid']

In [7]:
#Buliding the first layer (Convolution Layer) of the network
def build_layer_1():
    
    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
    return model

In [8]:
#Adding a convolution layer to the model(network)
def add_convolution(model):
    
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv)) 
    return model

In [9]:
#Adding a max pooling layer to the model(network)
def add_max_pooling(model,nb_pool):
    
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    return model

In [10]:
#Adding dropout to the model(network)
def add_droput(model, x):
    
    model.add(Dropout(x))
    return model

In [11]:
#Adding a flattening layer to the model(network), before adding a dense layer
def add_flatten(model):
    
    model.add(Flatten())
    return model

In [12]:
#Adding a dense layer to the model(network)
def add_dense(model,nodes):
    
    model.add(Dense(nodes))
    return model

In [13]:
#Adding a activation layer to the model(network)
def add_activation(model, activation):
    
    model.add(Activation(activation))
    return model

In [14]:
#The final step in building the model, compiling and fitting the model to the data
def build_layer_final(model):
    
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    compile_fit_score_model(model)

In [15]:
#Compiling, fitting and scoring the model
def compile_fit_score_model(model_final):
    
    model_final.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
    
    model_final.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,verbose=1,
              validation_data=(X_test, Y_test))
    
    score = model_final.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [20]:
#1.Initial convolution layer
def build_layers():
    
    np.random.seed(1338)
    
    model = build_layer_1()
    model = add_activation(model, 'relu')
    model = add_flatten(model)
    build_layer_final(model)

In [21]:
%timeit -n1 -r1 build_layers()

Train on 11918 samples, validate on 6000 samples
Epoch 1/12
11918/11918 [==============================] - 4s - loss: 0.1640 - acc: 0.9481 - val_loss: 0.0970 - val_acc: 0.9650
Epoch 2/12
11918/11918 [==============================] - 4s - loss: 0.0545 - acc: 0.9825 - val_loss: 0.0440 - val_acc: 0.9828
Epoch 3/12
11918/11918 [==============================] - 4s - loss: 0.0269 - acc: 0.9916 - val_loss: 0.0462 - val_acc: 0.9818
Epoch 4/12
11918/11918 [==============================] - 4s - loss: 0.0185 - acc: 0.9941 - val_loss: 0.0284 - val_acc: 0.9892
Epoch 5/12
11918/11918 [==============================] - 4s - loss: 0.0101 - acc: 0.9975 - val_loss: 0.0326 - val_acc: 0.9870
Epoch 6/12
11918/11918 [==============================] - 5s - loss: 0.0082 - acc: 0.9980 - val_loss: 0.0241 - val_acc: 0.9908
Epoch 7/12
11918/11918 [==============================] - 5s - loss: 0.0063 - acc: 0.9986 - val_loss: 0.0204 - val_acc: 0.9933
Epoch 8/12
11918/11918 [==============================] - 5s -

In [18]:
#1.Initial convolution layer, 2.Add convolution layer
def build_layers_2():
    np.random.seed(1338)
    
    model = build_layer_1()
    model = add_activation(model, 'relu')
    
    model = add_convolution(model)
    model = add_activation(model, 'relu')
    model = add_flatten(model)
    build_layer_final(model)

In [19]:
%timeit -n1 -r1 build_layers_2()

Train on 11918 samples, validate on 6000 samples
Epoch 1/12
11918/11918 [==============================] - 41s - loss: 0.1001 - acc: 0.9646 - val_loss: 0.0415 - val_acc: 0.9840
Epoch 2/12
11918/11918 [==============================] - 45s - loss: 0.0222 - acc: 0.9926 - val_loss: 0.0114 - val_acc: 0.9970
Epoch 3/12
11918/11918 [==============================] - 45s - loss: 0.0144 - acc: 0.9950 - val_loss: 0.0236 - val_acc: 0.9920
Epoch 4/12
11918/11918 [==============================] - 40s - loss: 0.0073 - acc: 0.9977 - val_loss: 0.0785 - val_acc: 0.9742
Epoch 5/12
11918/11918 [==============================] - 40s - loss: 0.0061 - acc: 0.9979 - val_loss: 0.0300 - val_acc: 0.9898
Epoch 6/12
11918/11918 [==============================] - 41s - loss: 0.0031 - acc: 0.9992 - val_loss: 0.0192 - val_acc: 0.9947
Epoch 7/12
11918/11918 [==============================] - 34s - loss: 0.0023 - acc: 0.9994 - val_loss: 0.0200 - val_acc: 0.9940
Epoch 8/12
11918/11918 [==============================]

In [20]:
#1.Initial convolution layer, 2.Convolution layer, 3.Add max pooling
def build_layers_3():
    np.random.seed(1338)
    
    model = build_layer_1()
    model = add_activation(model, 'relu')
    
    model = add_convolution(model)
    model = add_activation(model, 'relu')
    
    model = add_max_pooling(model)
    
    model = add_flatten(model)
    build_layer_final(model)

In [21]:
%timeit -n1 -r1 build_layers_3()

Train on 11918 samples, validate on 6000 samples
Epoch 1/12
11918/11918 [==============================] - 23s - loss: 0.1303 - acc: 0.9567 - val_loss: 0.1501 - val_acc: 0.9425
Epoch 2/12
11918/11918 [==============================] - 24s - loss: 0.0274 - acc: 0.9914 - val_loss: 0.0455 - val_acc: 0.9828
Epoch 3/12
11918/11918 [==============================] - 23s - loss: 0.0155 - acc: 0.9951 - val_loss: 0.0223 - val_acc: 0.9918
Epoch 4/12
11918/11918 [==============================] - 23s - loss: 0.0122 - acc: 0.9958 - val_loss: 0.0294 - val_acc: 0.9880
Epoch 5/12
11918/11918 [==============================] - 23s - loss: 0.0095 - acc: 0.9961 - val_loss: 0.0201 - val_acc: 0.9917
Epoch 6/12
11918/11918 [==============================] - 23s - loss: 0.0076 - acc: 0.9978 - val_loss: 0.0159 - val_acc: 0.9952
Epoch 7/12
11918/11918 [==============================] - 23s - loss: 0.0050 - acc: 0.9981 - val_loss: 0.0240 - val_acc: 0.9923
Epoch 8/12
11918/11918 [==============================]

In [22]:
#1.Initial convolution layer, 2.Convolution layer, 3.Max pooling, 4.Add dropout(0.25)
def build_layers_4():
    np.random.seed(1338)
    
    model = build_layer_1()
    model = add_activation(model, 'relu')
    
    model = add_convolution(model)
    model = add_activation(model, 'relu')
    
    model = add_max_pooling(model)
    model = add_droput(model,0.25)
    
    model = add_flatten(model)
    build_layer_final(model)

In [23]:
%timeit -n1 -r1 build_layers_4()

Train on 11918 samples, validate on 6000 samples
Epoch 1/12
11918/11918 [==============================] - 26s - loss: 0.1188 - acc: 0.9585 - val_loss: 0.1940 - val_acc: 0.9210
Epoch 2/12
11918/11918 [==============================] - 26s - loss: 0.0333 - acc: 0.9881 - val_loss: 0.0237 - val_acc: 0.9902
Epoch 3/12
11918/11918 [==============================] - 26s - loss: 0.0238 - acc: 0.9905 - val_loss: 0.0146 - val_acc: 0.9950
Epoch 4/12
11918/11918 [==============================] - 26s - loss: 0.0178 - acc: 0.9934 - val_loss: 0.0184 - val_acc: 0.9943
Epoch 5/12
11918/11918 [==============================] - 26s - loss: 0.0126 - acc: 0.9951 - val_loss: 0.0412 - val_acc: 0.9838
Epoch 6/12
11918/11918 [==============================] - 26s - loss: 0.0115 - acc: 0.9965 - val_loss: 0.0199 - val_acc: 0.9937
Epoch 7/12
11918/11918 [==============================] - 26s - loss: 0.0092 - acc: 0.9965 - val_loss: 0.0165 - val_acc: 0.9948
Epoch 8/12
11918/11918 [==============================]

In [24]:
#1.Initial convolution layer, 2.Convolution layer, 3.Max pooling, 4.Dropout(0.25), 5.Add dense layer
def build_layers_5():
    np.random.seed(1338)
    
    model = build_layer_1()
    model = add_activation(model, 'relu')
    
    model = add_convolution(model)
    model = add_activation(model, 'relu')
    
    model = add_max_pooling(model)
    model = add_droput(model,0.25)
    
    model = add_flatten(model)
    model = add_dense(model,128)
    model = add_activation(model, 'relu')
    
    build_layer_final(model)

In [25]:
%timeit -n1 -r1 build_layers_5()

Train on 11918 samples, validate on 6000 samples
Epoch 1/12
11918/11918 [==============================] - 28s - loss: 0.1335 - acc: 0.9605 - val_loss: 0.2473 - val_acc: 0.9020
Epoch 2/12
11918/11918 [==============================] - 27s - loss: 0.0285 - acc: 0.9895 - val_loss: 0.2526 - val_acc: 0.9238
Epoch 3/12
11918/11918 [==============================] - 27s - loss: 0.0187 - acc: 0.9930 - val_loss: 0.0439 - val_acc: 0.9847
Epoch 4/12
11918/11918 [==============================] - 27s - loss: 0.0105 - acc: 0.9961 - val_loss: 0.0279 - val_acc: 0.9900
Epoch 5/12
11918/11918 [==============================] - 27s - loss: 0.0114 - acc: 0.9961 - val_loss: 0.0166 - val_acc: 0.9948
Epoch 6/12
11918/11918 [==============================] - 27s - loss: 0.0061 - acc: 0.9977 - val_loss: 0.0299 - val_acc: 0.9908
Epoch 7/12
11918/11918 [==============================] - 27s - loss: 0.0044 - acc: 0.9985 - val_loss: 0.0261 - val_acc: 0.9918
Epoch 8/12
11918/11918 [==============================]

In [26]:
#1.Initial convolution layer, 2.Convolution layer, 3.Max pooling, 4.Dropout(0.25), 5.Dense layer, 6.Add dropout(0.5)
def build_layers_6():
    np.random.seed(1338)
    
    model = build_layer_1()
    model = add_activation(model, 'relu')
    
    model = add_convolution(model)
    model = add_activation(model, 'relu')
    
    model = add_max_pooling(model)
    model = add_droput(model,0.25)
    
    model = add_flatten(model)
    model = add_dense(model,128)
    model = add_activation(model, 'relu')
    
    model = add_droput(model,0.5)
    
    build_layer_final(model)

In [27]:
%timeit -n1 -r1 build_layers_6()

Train on 11918 samples, validate on 6000 samples
Epoch 1/12
11918/11918 [==============================] - 27s - loss: 0.1442 - acc: 0.9535 - val_loss: 0.1519 - val_acc: 0.9440
Epoch 2/12
11918/11918 [==============================] - 27s - loss: 0.0346 - acc: 0.9883 - val_loss: 0.0294 - val_acc: 0.9907
Epoch 3/12
11918/11918 [==============================] - 27s - loss: 0.0204 - acc: 0.9930 - val_loss: 0.0191 - val_acc: 0.9932
Epoch 4/12
11918/11918 [==============================] - 27s - loss: 0.0154 - acc: 0.9939 - val_loss: 0.0455 - val_acc: 0.9820
Epoch 5/12
11918/11918 [==============================] - 27s - loss: 0.0121 - acc: 0.9956 - val_loss: 0.0263 - val_acc: 0.9912
Epoch 6/12
11918/11918 [==============================] - 27s - loss: 0.0113 - acc: 0.9960 - val_loss: 0.0191 - val_acc: 0.9935
Epoch 7/12
11918/11918 [==============================] - 27s - loss: 0.0089 - acc: 0.9962 - val_loss: 0.0209 - val_acc: 0.9945
Epoch 8/12
11918/11918 [==============================]

In [28]:
#(Parallel(n_jobs = -1)(delayed(function)()for function in (build_layers,build_layers_2)))      